# Wedding Seating Allocation Algorithm

This script tries to achieve the best allocation of guests possible to a given number of tables in terms of party mood. It needs several inputs:

- s: maximum number of guests per table or an array of length t (number of tables) holding the seats available at each table like [8,9,8,8]
- m: matrix of guests and closeness indicator for each pair of guests, like so:

| Guests 	| Bob 	| Alice 	| Kenny 	| Celine 	|
|--------	|-----	|-------	|-------	|--------	|
| Bob    	| NA   	| 9     	| -9    	| NA     	|
| Alice  	| NA  	| NA     	| 1     	| NA     	|
| Kenny  	| NA  	| NA    	| NA     	| 9     	|
| Celine 	| NA  	| NA    	| NA    	| NA      	|

Inside the guest matrix, the higher the value the closer a pair is (negative signifies dislike). NAs and 0s mark pairs who either do not know each other or the impossible pair of the same person. We asusme mutual closeness here, meaning only the upper (or lower) half of the matrix has to be filled, i.e. there is only one look-up closeness value for each pair.
So in this toy example Bob & Alice are together as well as Kenny & Celine. Bob & Kenny though are arch-enemies and Alice knows Kenny without having any strong feelings.

## Preparations

In [1]:
# modules
import pandas as pd
import numpy as np
import random as rd

### Closeness matrix **m**

In [2]:
m = pd.read_csv('./data/toBeSeated.csv', index_col=0)

In [3]:
m.head()

,Jorg,Carola,Ingrid,Alina,Alex,Pirmin,Sophie,Helga,Anja,Eckhard,...,Melanie K,Uli,Kim,Anna-Luisa,Marcel,Jakob,Jeannine,Burak,Lukas,Merle
Jorg,0.0,99.0,-99.0,-99.0,-99.0,-99.0,-99.0,1.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Carola,NaN,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ingrid,NaN,NaN,0.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alina,NaN,NaN,NaN,0.0,99.0,5.0,5.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alex,NaN,NaN,NaN,NaN,0.0,5.0,5.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# copy upper triangle to lower triangle
for i in range(len(m)):
    for j in range(i, len(m)):
        m[m.columns[i]][j] = m[m.columns[j]][i]

In [5]:
# fill up 0s
m.fillna(value=0, inplace=True)

In [6]:
m.head()

,Jorg,Carola,Ingrid,Alina,Alex,Pirmin,Sophie,Helga,Anja,Eckhard,...,Melanie K,Uli,Kim,Anna-Luisa,Marcel,Jakob,Jeannine,Burak,Lukas,Merle
Jorg,0.0,99.0,-99.0,-99.0,-99.0,-99.0,-99.0,1.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Carola,99.0,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ingrid,-99.0,-99.0,0.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alina,-99.0,-99.0,5.0,0.0,99.0,5.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alex,-99.0,-99.0,5.0,99.0,0.0,5.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### seats and tables **s**

In [7]:
print('Number of people to be seated: '+str(len(m)))

Number of people to be seated: 36


In [8]:
s = [9, 9, 9, 9]
print('Number of seats over all tables: '+str(sum(s)))

Number of seats over all tables: 36


## Initial seating

In [16]:
seats_names = []
table_satisf = []
for i in range(len(s)):
    n_start = sum(s[:i])
    n_end = sum(s[:(i+1)])
    
    names = m.columns[n_start:n_end].tolist()
    fit = pd.DataFrame({'score':m[m.index.isin(names)][names].sum(axis=1)})
    
    seats_names.append(fit)
    table_satisf.append(sum(fit.score))

In [17]:
table_satisf

[-1244.0, 666.0, 926.0, 1012.0]

In [18]:
print('Total satisfaction is: '+str(sum(table_satisf)))

Total satisfaction is: 1360.0


## Do 1000 pairwise changes

In [19]:
# extract 2 people at random and let them trade seats if the sum of the table scores improves
ts = list(range(4))
for _ in range(10000):
    
    # random table
    t1,t2 = rd.sample(set(ts), 2)
    
    # random person
    s1=rd.choice(seats_names[t1].index)
    s2=rd.choice(seats_names[t2].index)
    
    # new tables
    n1 = list(seats_names[t2].drop(s2).index); n1.append(s1)
    n2 = list(seats_names[t1].drop(s1).index); n2.append(s2)
    
    # new table scores
    newt1 = pd.DataFrame({'score':m[m.index.isin(n1)][n1].sum(axis=1)})
    newt2 = pd.DataFrame({'score':m[m.index.isin(n2)][n2].sum(axis=1)})
    
#     if (newt1.sum()[0]>seats_names[t1].sum()[0]) & (newt2.sum()[0]>seats_names[t2].sum()[0]):
    if (newt1.sum()[0]+newt2.sum()[0])>(seats_names[t1].sum()[0]+seats_names[t2].sum()[0]):
        seats_names[t1] = newt1
        seats_names[t2] = newt2
    else:
        pass

In [20]:
satisf = 0
for i in range(len(seats_names)):
    i_satisf = sum(seats_names[i].score)
    print('Table '+str(i+1)+' satisfaction: '+str(i_satisf))
    satisf+=i_satisf

Table 1 satisfaction: 736.0
Table 2 satisfaction: 1010.0
Table 3 satisfaction: 872.0
Table 4 satisfaction: 900.0


In [21]:
print('New total satisfaction is: '+str(satisf))

New total satisfaction is: 3518.0


In [15]:
seats_names

[        score
 Chris    99.0
 Mark     99.0
 Chae      9.0
 Erik    107.0
 Sarah   100.0
 Lorenz  111.0
 Stef    111.0
 Cruz     19.0
 Marcel   17.0,          score
 Ingrid    25.0
 Alina    119.0
 Alex     119.0
 Pirmin   119.0
 Sophie   119.0
 Helga     25.0
 Anja      99.0
 Eckhard   99.0
 Burak      0.0,            score
 Juri        40.0
 Tommy      118.0
 Tanja      116.0
 Adler      118.0
 Melanie A  116.0
 Roman      118.0
 Sofia      114.0
 Toby       118.0
 Melanie K  114.0,             score
 Jorg         99.0
 Carola       99.0
 Uli         198.0
 Kim         198.0
 Anna-Luisa  198.0
 Jakob        99.0
 Jeannine     99.0
 Lukas        99.0
 Merle        99.0]